# Processing and cleaning data
The phases of this part are divided in several sections. Coding styles and techniques vary from one section to fit the specific purpose and according to the skill area of the project member who was writing it.

In [46]:
import pandas as pd
import numpy as np
from datetime import timedelta
from tqdm import tqdm

In [47]:
td_dataset = pd.read_stata("data/td_ita.dta")  # time diaries dataset
demo_dataset = pd.read_stata("data/data4diarynew_ITA.dta") # demographics dataset
stepDetector = pd.read_csv("data/stepdetectorevent.csv") # step counter dataset

## Base - level cleaning: which users are fit for analysis.
An eligible user must not be a drop out, so there is data about their life for every day of the first few weeks of the experiment. 
For "total count" statistical measures other operations are performed. For example, to count how much of the user's day was spent on doing sports, there need to be null answers in the data to signify the times in which a user was inactive that will be summed to the other answers; then we obtain a complete activity dataset where the number of rows doing "Sports" equate exactly how much time each user has spent on Sports. 

**Purpose**: obtaining a list of user IDs that are fit for analysis, and filling missing timestamp data with null answers. 

**Process**: for each user, an algorithm called "gantt_data" checks if the user's data is consistent or not. Only users who pass this test are kept. For each of those, missing data is found and filled with nulle answers.

In [48]:
# Users who did sport AT LEAST once will be submitted to this algorithm.



time_accurate = td_dataset[(td_dataset.date_not.dt.month == 11 
                         ) & (td_dataset.date_not.dt.day >= 13
                             ) & (td_dataset.date_not.dt.day <= 30)]

once_active = time_accurate[time_accurate['what'] == 'Sport'].id.unique()

once_active = time_accurate[time_accurate['what'] == 'Sport'].id.unique()
IDs = pd.Series(time_accurate.id.unique()).astype(int) 
IDs = IDs[IDs.isin(once_active)]

user_activity = dict()
for ID in tqdm(IDs):
    id_act = time_accurate[time_accurate.id == ID]
    user_activity[ID] = list(id_act.date_not.dt.day.sort_values().unique())
print(f'At first there were about {time_accurate.id.unique().shape[0]} users. Now: N = ',len(user_activity.keys())) 
print("User dictionary with sorted unique user-specific observation days is done.")

def gantt_data_org(k, v):
    
    ''' 
    gantt_data is named after the Gantt chart.
    S is the Series of days (regardless of when they start).
    '''
    
    tot = 0
    start = np.nan
    end = np.nan
    consecutive = False
    
    if len(v):  #if it's not empty, it will check for continuity
        tot = (pd.to_datetime(v[-1]) - pd.to_datetime(v[0])).days
        start, end = v[0], v[-1]
        
        for i in range(len(v)-1):
            v[i]  #current
            v[i+1] #next
            difference = (pd.to_datetime(v[i+1]) - pd.to_datetime(v[i])).days
            # if there is more than two days of difference it means that there was more than one
            # day in the data with no observations, and it is unlikely it's consistent data.
            if difference >= 2:    
                consecutive = False
                break
            else:
                consecutive = True
                
    return pd.DataFrame([[k, start, end, tot, consecutive]], 
                        columns=['id', 'start', 'finish', 'tot', 'cons'])

gantt_data = pd.DataFrame(columns=["id", "start", "finish", "tot", "cons"])

for ID, days in user_activity.items():
    res = gantt_data_org(ID, days)
    gantt_data = pd.concat([gantt_data, res], ignore_index=True)
print("The test is finished.")
print("We end up with: N = ", gantt_data.id.unique().shape[0])
to_keep = gantt_data[gantt_data['cons'] == True].id

sport_eve = time_accurate[time_accurate.id.isin(to_keep)]

100%|██████████| 128/128 [00:01<00:00, 105.06it/s]


At first there were about 241 users. Now: N =  128
User dictionary with sorted unique user-specific observation days is done.
The test is finished.
We end up with: N =  128


In [49]:
print(user_activity[0])
print(user_activity[1])
#and so on.

[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]


Everyone who has done sport once (128 people) are also people who never dropped out from the experiment. They also all have observations ranging from 13th of November to 30th of November, as shown in the dictionary _user activity_.

In [50]:
print(f'Initial count of rows: {sport_eve.shape[0]:,}')

def today_or_yesterday(t): 

    '''
    This function labels the row with the day it belongs to using circadian time. 
    If an observation falls in the time range from midnight to 5 a.m. it is 
    classified as belonging to the precedent calendar day as we humans would perceive it. 
    Why 5 a.m.? It is where more than 99% of the users go to sleep or 
    wake up - which means that their day ended or started, 
    and their perception of which day it is has shifted by one.
    Otherwise, it's the same day as indicated the timestamp. 
    '''
    if t.hour < 5 and t.day > 13:
        return days[t.day - 1]
    else:
        return days[t.day]

days = dict()
for i in range(sport_eve.date_not.dt.day.min(), sport_eve.date_not.dt.day.max() + 1):
    days[i] = i - sport_eve.date_not.dt.day.min()  

min_range = pd.date_range(start="2020-11-13 00:00:00", end="2020-11-30 23:59:59", freq='30T')   
#notifications were fired every half hour and such must be the timing between null events
results = pd.DataFrame()
for user in tqdm(sport_eve.id.unique()):
    # each user data is gathered here
    subset = sport_eve[sport_eve.id == user]
    # an empty dataset is created with range
    complete_data = pd.DataFrame({'date_not': min_range})

    # the complete dataset gets the subset rows added at its bottom. 
    merged_df = pd.concat([complete_data, subset])
    #the id column, being a costant in each iteration, is filled with the user passed throug the iteration
    merged_df['id'] = merged_df['id'].fillna(user)

    #the merged dataset gets values sorted: for one timestamp there is a NaN and possibly a real activity from timediary
    # here below they are sorted with NaN being put first, and then dropped in the following line
    merged_df = merged_df.sort_values(by="what", na_position='first', ascending=True)
    merged_df = merged_df.drop_duplicates(keep='last', subset='date_not')

    merged_df['what'] = merged_df['what'].fillna('No information')
    #merged_df['what'] = merged_df['what'].astype(str)

    error = 'Some mistake occurred. \nThe amount of observation time is different than expected.'
    
    if merged_df.date_not.min() != complete_data.date_not.min() or merged_df.date_not.max() != complete_data.date_not.max():
        print(f"\n{error}\nUser: {user}, dates: {merged_df.date_not.min(), merged_df.date_not.max()}\n")
    if merged_df.shape[0] != complete_data.shape[0]: 
        print(f"\n{error}\nUser: {user}, length: {merged_df.shape[0]}, expected: {complete_data.shape[0]}\n")
    results = pd.concat([results, merged_df], ignore_index=True)
results['day'] = [today_or_yesterday(time) for time in results.date_not]
results.id = results.id.astype(int)
print(f'Finished. Total count of rows: {results.shape[0]:,}')

Initial count of rows: 108,800


100%|██████████| 128/128 [00:15<00:00,  8.32it/s]


Finished. Total count of rows: 110,592


Quick math to explain why exactly $110,592$ rows:

Each day has 48 half hours - corresponding to the times a notification was fired. 
Each user has 18 days of observations in the complete dataset. 
There are 128 users.
$48*18*128 =110,592 $

The "complete but empty" dataset called _complete_data_ is used to attach onto it each user's actual answers. Then, if for the same timestamp there are two observations, the empty one gets dropped to make space for the actual answer. There are only 2,592 differences, because the users did not classify them as some specific activity when they weren't answering the phone.

In [51]:
#MODIFIED DATASET

check = results
check['day'] = check['date_not'].dt.day 
total = []
for day in check['day'].unique():
    total.append(sum(check[check['day']==day][['what']].value_counts().sort_values()))
#Further check: each day should have the same amount of observations. 
print(total)
print()
print('Completed dataset')
check.what.value_counts().sort_values(ascending=False) #you can see that "No information" receives more observations than in the original data.

[6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144]

Completed dataset


what
No information                                    26752
Sleeping                                          18069
Study/work group                                  12036
I will go to sleep                                 7388
Eating                                             6097
Watching TV, video, YouTube, etc.                  5383
Lecture/seminar/conference/university meeting      4302
Expired                                            3404
Personal care                                      2671
Did not do anything special                        2271
Social life                                        2213
Cooking, Food preparation & management             1828
Games                                              1692
Household and family care                          1628
Sport                                              1433
Rest/nap                                           1372
Phone/Video calling                                1326
Walking                                    

In [52]:
#ORIGINAL DATA
check = sport_eve.copy()
check['day'] = check['date_not'].dt.day
total = []
for day in check['day'].unique():
    total.append(sum(check[check['day']==day][['what']].value_counts().sort_values()))
print(total)
check.what.value_counts().sort_values(ascending=False)  

[6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 6144, 4352]


what
No information                                    24960
Sleeping                                          18069
Study/work group                                  12036
I will go to sleep                                 7388
Eating                                             6097
Watching TV, video, YouTube, etc.                  5383
Lecture/seminar/conference/university meeting      4302
Expired                                            3404
Personal care                                      2671
Did not do anything special                        2271
Social life                                        2213
Cooking, Food preparation & management             1828
Games                                              1692
Household and family care                          1628
Sport                                              1433
Rest/nap                                           1372
Phone/Video calling                                1326
Walking                                    

### Saving the data for later analysis

In [53]:
%%timeit
results.to_csv('data/sport_eventformat.csv')

5.05 s ± 215 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Sport data - from events to sessions
In order to use perform specific analyses, the data must fit specific machine-readable standards. In this section for later purposes (in other notebooks).

**Purpose**: obtain duration of sport activity, associations between location and activity, between human company and activity, and finally type of activity. 

**Process**: to understand the way people have answered "Sport", the events of said activity are grouped with each other when they are adjacent. Other code re-categorization are also performed for simplicity. Other techniques are used to model the data in order to obtain the associations.

In [54]:
# Session Duration
td_cleaned = results[(results["what"]=="Sport") | (results["what"]=="I will participate in sports activities") |(results["what"]=="Walking")]

new_td = td_cleaned
new_td['date_not'] = pd.to_datetime(new_td['date_not'])
new_td['date_dur'] = new_td['date_not'] - timedelta(hours = 5)

new_td = new_td.sort_values('date_dur') # sort the dataframe according to "date_dur"
new_td['time_diff'] = new_td['date_dur'].diff() # compute the time difference between adjacent rows

new_session = (new_td['time_diff'] > pd.Timedelta(minutes=30)) | new_td['time_diff'].isnull() # search for new start session
new_td['session'] = new_session.cumsum() # Creazione di un nuovo identificatore di sessione basato su new_session

collapsed_sessions = new_td.groupby(['id', 'session']).agg(
    start_time=('date_not', 'first'),
    end_time=('date_not', 'last'),
    duration=('date_dur', lambda x: ((x.max() - x.min()).total_seconds() / 60)+30)
).reset_index()

collapsed_sessions = collapsed_sessions.drop(columns='session')

# merge the two datasets
td_sessions = td_cleaned.merge(collapsed_sessions, left_on=['id', 'date_not'], right_on=['id', 'start_time']) # inner join of the two datasets

/var/folders/by/k1qr78752n52b1dz4bq0f23h0000gn/T/ipykernel_82084/1554253985.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_td['date_not'] = pd.to_datetime(new_td['date_not'])
/var/folders/by/k1qr78752n52b1dz4bq0f23h0000gn/T/ipykernel_82084/1554253985.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_td['date_dur'] = new_td['date_not'] - timedelta(hours = 5)


### Some skimming
In this subsection we evaluate if each session is above 30 minutes of duration (even walking, which may be considered a sport when done above a certain threshold). 

In [55]:
print(f"Initial number of cases: {td_sessions.shape[0]}")

# obtain the indexes of the rows satisfying condition 1
to_remove1 = list(td_sessions[(td_sessions['what']=='Walking') & (td_sessions['duration']==30.0)].index)

# remove these cases
td_sessions = td_sessions.drop(index=to_remove1)
print(f"Number of cases after first cleaning: {td_sessions.shape[0]}")

# obtain the indexes of the rows satisfying condition 2
places = [ 'Supermarket …', 'Street markets', 'Grocery Shop', 'University Library', 'Other university place']
to_remove2 = list(td_sessions[(td_sessions['what']=='Walking') & (td_sessions['where'].isin(places))].index)

# remove these cases
td_cleaned = td_sessions.drop(index=to_remove2)
print(f"Number of cases after second cleaning: {td_sessions.shape[0]}")

Initial number of cases: 1028
Number of cases after first cleaning: 864
Number of cases after second cleaning: 864


In [56]:
# location of sport activity variable
td_cleaned['where recoded'] = td_cleaned['where'].replace(['Home apartment /room', 'Weekend home or holiday apartment', 'House (friends others)', 'Relatives Home', 'Home garden/patio/courtyard',
                                                           'Another indoor place', 'Gym, swimming pool, Sports centre …', 'Other university place', 'Countryside/mountain/hill/beach', 'In the street', 'Another outdoor place', 'Café, pub, bar', 'Shops, shopping centres', 'Not answer'],
                                                          ['indoor', 'indoor', 'indoor', 'indoor', 'indoor',
                                                           'outdoor', 'outdoor', 'outdoor', 'outdoor', 'outdoor', 'outdoor', 'outdoor', 'outdoor', 'not specified'])
td_cleaned['where recoded'] = td_cleaned['where recoded'].cat.remove_unused_categories()


# company during sport activity variable
td_cleaned['withw recoded'] = td_cleaned['withw'].replace(['Partner', 'Friend(s)', 'Relative(s)', 'Roommate(s)', 'Other', 'Colleague(s)',
                                                           'Classmate(s)',
                                                           'Alone',
                                                           'Not answer'],
                                                          ['company', 'company', 'company', 'company', 'company', 'company', 'company',
                                                           'alone',
                                                           'not specified'])
td_cleaned['withw recoded'] = td_cleaned['withw recoded'].cat.remove_unused_categories()


# type of sport activity variable
td_cleaned['sport recoded'] = td_cleaned['sport'].replace(['Walking, Trekking, and hiking',
                                                           'Jogging and running',
                                                           'Cycling, skiing, and skating', 'Ball games', 'Other outdoor activities',
                                                           'Gymnastics and fitness',
                                                           'Water sports', 'Other indoor activities'
                                                           ],
                                                          ['Walking, Trekking, and hiking',
                                                           'Jogging and running',
                                                           'Outdoor activities', 'Outdoor activities', 'Outdoor activities',
                                                           'Gymnastics and fitness',
                                                           'Other indoor activities', 'Other indoor activities'
                                                           ])

# adding the category Walking to the sport activity
td_cleaned['sport recoded'] = td_cleaned['sport recoded'].cat.add_categories('Walking')
td_cleaned.loc[td_cleaned['what'] == 'Walking', 'sport recoded'] = 'Walking'
td_cleaned['sport recoded'] = td_cleaned['sport recoded'].cat.remove_unused_categories()

# useful conversions
td_cleaned['id'] = td_cleaned['id'].astype(int)
demo_dataset['userid'] = demo_dataset['userid'].astype(int)

# check how many missing values we have in the sport sessions (the 'not specified')
missing1 = list(td_cleaned[(td_cleaned['where recoded'] =='not specified') & (td_cleaned['withw recoded'] =='not specified')].index)
missing2 = list(td_cleaned[(td_cleaned['where recoded'] =='not specified') | (td_cleaned['withw recoded'] =='not specified')].index)

# since we have few missing values we decide to remove them
all_missing = list(set(missing1+missing2))
td_cleaned = td_cleaned.drop(index=all_missing)
print(f"the final dataset has size {td_cleaned.shape[0]}")

# selec only the variables we want to keep
new_dataset = td_cleaned[['id','sport recoded', 'withw recoded', 'where recoded']]

# merge with the socio-demographic dataframe
final_dataset = new_dataset.merge(demo_dataset, left_on='id', right_on='userid', how='left')

final_dataset = final_dataset[[ 'sport recoded', 'where recoded', 'withw recoded','degree', 'Dep_UNITN', 'w1_A10UNITN', 'w1_A01']]
final_dataset.columns = ['type of activity', 'location', 'company', 'degree', 'department', 'neighborhood', 'sex']


the final dataset has size 844


### Saving the data for later analysis
Data organized in sessions such as above is saved for analysis in other modules. 

In [57]:
%%timeit
final_dataset.to_csv('data/sport_sessionformat.csv')

6.66 ms ± 752 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## sport activity definition
We use the previous dataset of sport session to define users as active or inactive according to the number of activities done in the two weeks

In [58]:
active_data = {'userid':td_cleaned['id'],
               'session': np.array(1)}

active_users_df = pd.DataFrame(active_data)

sport_sessions = active_users_df.groupby('userid')['session'].sum().reset_index() # we count the number of sessions done by each user
mean_session = sport_sessions['session'].mean() # compute the mean
print(f"the mean number of session in the two weeks was {int(mean_session)}")

sport_activity = sport_sessions.groupby('userid')['session'].sum().reset_index().rename(columns={'session': 'sessions done'})

# we define users as active or very active based on the mean of sport sessions
sport_activity['sport activity'] = pd.qcut(sport_activity['sessions done'], q=[0, 1/2, 1], labels=['active', 'very active'])

# check: there should be the 128 users who did at least one time sport
sport_activity.shape

# we still need the inactive category that we'll add when there is the complete dataset

the mean number of session in the two weeks was 6


(128, 3)

## Step detector data - from events to sessions
**Purpose:** possess knowledge on sensor data - which can be arguably more precise than time diary answers hours after the event. 

**Process:** the sensor data is in the form of events; every sharp 30 minutes worth of events is aggregated and will result in one row. 

In [59]:
stepDetector['timestamp'] = pd.to_datetime(stepDetector['timestamp'], format='%Y%m%d%H%M%S%f')
stepDetector1min = stepDetector.copy()
stepDetector30min = stepDetector.copy()

stepDetector1min['rounded_timestamp'] = stepDetector1min['timestamp'].dt.floor('T')
result1m = stepDetector1min.groupby(['userid', 'day', 'rounded_timestamp']).size().reset_index(name="step_count")


stepDetector30min['rounded_timestamp'] = stepDetector30min['timestamp'].dt.floor('30T')
result30min = stepDetector30min.groupby(['userid', 'day', 'rounded_timestamp']).size().reset_index(name="step_count")

In [60]:
result30min = result30min[result30min.rounded_timestamp <= "2020-11-30 23:59:59"].copy()

In [61]:
print(f'Initial count of rows: {result30min.shape[0]:,}')

days = dict()
for i in range(result30min.rounded_timestamp.dt.day.min(), result30min.rounded_timestamp.dt.day.max() + 1):
    days[i] = i - result30min.rounded_timestamp.dt.day.min()  

min_range = pd.date_range(start="2020-11-10 00:00:00", end="2020-11-30 23:59:59", freq='30T')   
#notifications were fired every half hour and such must be the timing between null events
results = pd.DataFrame()
for user in tqdm(result30min.userid.unique()):
    # each user data is gathered here
    subset = result30min[result30min.userid == user]
    # an empty dataset is created with range
    complete_data = pd.DataFrame({'rounded_timestamp': min_range})

    # the complete dataset gets the subset rows added at its bottom. 
    merged_df = pd.concat([complete_data, subset])
    #the id column, being a costant in each iteration, is filled with the user passed throug the iteration
    merged_df['userid'] = merged_df['userid'].fillna(user)

    #the merged dataset gets values sorted: for one timestamp there is a NaN and possibly a real activity from timediary
    # here below they are sorted with NaN being put first, and then dropped in the following line
    merged_df = merged_df.sort_values(by="step_count", na_position='first', ascending=True)
    merged_df = merged_df.drop_duplicates(keep='last', subset='rounded_timestamp')

    merged_df['step_count'] = merged_df['step_count'].fillna(0) #NaN will be replaced with 0. 

    error = 'Some mistake occurred. \nThe amount of observation time is different than expected.'
    
    if merged_df.rounded_timestamp.min() != complete_data.rounded_timestamp.min() or merged_df.rounded_timestamp.max() != complete_data.rounded_timestamp.max():
        print(f"\n{error}\nUser: {user}, dates: {merged_df.rounded_timestamp.min(), merged_df.rounded_timestamp.max()}\n")
    if merged_df.shape[0] != complete_data.shape[0]: 
        print(f"\n{error}\nUser: {user}, length: {merged_df.shape[0]}, expected: {complete_data.shape[0]}\n")
    results = pd.concat([results, merged_df], ignore_index=True)
    
#results['day'] = [today_or_yesterday(time) for time in results.date_not]
results.userid = results.userid.astype(int)
results = results.rename(columns={"userid": "id"}).copy()
print(f'Finished. Total count of rows: {results.shape[0]:,}')

Initial count of rows: 22,587


100%|██████████| 126/126 [00:01<00:00, 114.27it/s]

Finished. Total count of rows: 127,008


### Saving the data for later analysis

In [62]:
%%timeit
result1m.to_csv("data/stepDetector_1min.csv")
result30min.to_csv("data/stepDetector_30min.csv")

1.11 s ± 56.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [63]:
result30min

,userid,day,rounded_timestamp,step_count
0,0,20201114,2020-11-14 08:30:00,39
1,0,20201114,2020-11-14 12:00:00,148
2,0,20201115,2020-11-15 09:30:00,5
3,0,20201117,2020-11-17 12:00:00,81
4,0,20201117,2020-11-17 16:30:00,13
...,...,...,...,...
33964,265,20201123,2020-11-23 20:30:00,177
33965,265,20201124,2020-11-24 15:30:00,15
33966,265,20201124,2020-11-24 16:30:00,30
33967,265,20201124,2020-11-24 19:00:00,142


## step activity definition
we define users whose step detector sensor was switched with different levels of activity based on the step taken

In [64]:
# pre-processing for the step dataset
ids = result30min['userid'].unique()
#result30min = result30min.drop("Unnamed: 0", axis=1)

# step count
df = result30min.copy()
daily_step_counts = df.groupby(['userid', 'day'])['step_count'].sum().reset_index()

steps_median = daily_step_counts['step_count'].median()

step_activity_df = daily_step_counts.groupby('userid')['step_count'].sum().reset_index().rename(
    columns={'step_count': 'tot daily steps'})

step_activity_df['step activity'] = pd.qcut(step_activity_df['tot daily steps'], q=[0, 1 / 3, 2 / 3, 1],
                                            labels=['not very active', 'active', 'very active'])


## merge process
we want to put all together in a single dataset to compare sport active users with step active users

In [65]:
# merge the demo dataset with the step activity dataset
demo_dataset['userid'] = pd.to_numeric(demo_dataset['userid'])
first_dataset = demo_dataset.merge(step_activity_df, right_on="userid", left_on="userid", how="left")

# merge the dataset with the sport activity dataset
complete_dataset = first_dataset.merge(sport_activity, right_on="userid", left_on="userid", how="left")

In [66]:
complete_dataset

,token,w1_idpilot,userid,pilot,w1_A01,nationality,department,Dep_UNITN,cohort,degree,...,Environmental,Spiritual,w1_A03,w1_A04UNITN,w1_A09UNITN,w1_A10UNITN,tot daily steps,step activity,sessions done,sport activity
0,D2XizUqSSzXNNMx,UNITN,0,UNITN,Female,Kazakh,Engineering and Applied Sciences,Engineering and Applied Sciences,24,MSc,...,58.333332,43.75,Kazakh,Industrial Engineering,Trento,Trento South (Bolghera/Clarina/San Bartolomeo/...,872.0,not very active,6.0,active
1,0qV69OJ7wQPIzz5,UNITN,1,UNITN,Female,Italian,Humanities,Humanities,23,BSc,...,58.333332,56.25,Italian,Humanities,In another city away from where the university...,NaN,NaN,NaN,7.0,very active
2,2cOaR8EzHKmtVhr,UNITN,10,UNITN,Male,Italian,Agricultural,Agricultural,20,BSc,...,83.333336,75.00,Italian,Center Agriculture Food Environment,Trento,Center (from Cristo Re/San Martino to Fersina),7.0,not very active,2.0,active
3,ZhAEBUHouZHiUae,UNITN,100,UNITN,Female,Italian,Social Sciences,Social Sciences,19,BSc,...,58.333332,37.50,Italian,Sociology and Social Research,Close to the city where the university is loca...,NaN,59277.0,very active,4.0,active
4,cXHpqdIIb4aRKUH,UNITN,101,UNITN,Female,Italian,Law,Law,24,MSc,...,NaN,NaN,Italian,Faculty of Law,Trento,Center (from Cristo Re/San Martino to Fersina),NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,1SaWdxnApikIWlR,UNITN,95,UNITN,Female,Italian,Social Sciences,Social Sciences,21,BSc,...,83.333336,75.00,Italian,Psychology and Cognitive Science,Rovereto,NaN,3994.0,not very active,4.0,active
245,I8ggVucB5iLIW7C,UNITN,96,UNITN,Male,Italian,Social Sciences,Social Sciences,20,BSc,...,NaN,NaN,Italian,Sociology and Social Research,In another city away from where the university...,NaN,43811.0,very active,NaN,NaN
246,XpFgWbI8bvt4REU,UNITN,97,UNITN,Male,Italian,Engineering and Applied Sciences,Engineering and Applied Sciences,22,MSc,...,75.000000,56.25,Italian,Information Engineering and Computer Science,Trento,East hill (Mesiano/Povo/Oltrecastello/San Dona...,23515.0,active,5.0,active
247,nq8et8Sx7XXJ3P7,UNITN,98,UNITN,Male,Italian,Natural Sciences,Natural Sciences,22,BSc,...,75.000000,37.50,Italian,Physics,Trento,Trento South (Bolghera/Clarina/San Bartolomeo/...,NaN,NaN,12.0,very active


In [67]:
# sport activity reclassification: we already know sport active users, so all the remaining users should be considered as inactive
complete_dataset['sport activity'] = pd.Categorical(complete_dataset['sport activity'])
complete_dataset['sport activity'] = complete_dataset['sport activity'].cat.set_categories(['very active', 'active', 'inactive'])
complete_dataset['sport activity'] = complete_dataset['sport activity'].fillna("inactive")

# check that everything went right
print(f"from the sport activity dataset of {complete_dataset.shape[0]} users, the inactive ones are {complete_dataset[complete_dataset['sport activity']!='inactive'].shape[0]}, while the actives are {complete_dataset[complete_dataset['sport activity']=='inactive'].shape[0]}")

from the sport activity dataset of 249 users, the inactive ones are 126, while the actives are 123


In [68]:
complete_dataset

,token,w1_idpilot,userid,pilot,w1_A01,nationality,department,Dep_UNITN,cohort,degree,...,Environmental,Spiritual,w1_A03,w1_A04UNITN,w1_A09UNITN,w1_A10UNITN,tot daily steps,step activity,sessions done,sport activity
0,D2XizUqSSzXNNMx,UNITN,0,UNITN,Female,Kazakh,Engineering and Applied Sciences,Engineering and Applied Sciences,24,MSc,...,58.333332,43.75,Kazakh,Industrial Engineering,Trento,Trento South (Bolghera/Clarina/San Bartolomeo/...,872.0,not very active,6.0,active
1,0qV69OJ7wQPIzz5,UNITN,1,UNITN,Female,Italian,Humanities,Humanities,23,BSc,...,58.333332,56.25,Italian,Humanities,In another city away from where the university...,NaN,NaN,NaN,7.0,very active
2,2cOaR8EzHKmtVhr,UNITN,10,UNITN,Male,Italian,Agricultural,Agricultural,20,BSc,...,83.333336,75.00,Italian,Center Agriculture Food Environment,Trento,Center (from Cristo Re/San Martino to Fersina),7.0,not very active,2.0,active
3,ZhAEBUHouZHiUae,UNITN,100,UNITN,Female,Italian,Social Sciences,Social Sciences,19,BSc,...,58.333332,37.50,Italian,Sociology and Social Research,Close to the city where the university is loca...,NaN,59277.0,very active,4.0,active
4,cXHpqdIIb4aRKUH,UNITN,101,UNITN,Female,Italian,Law,Law,24,MSc,...,NaN,NaN,Italian,Faculty of Law,Trento,Center (from Cristo Re/San Martino to Fersina),NaN,NaN,NaN,inactive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,1SaWdxnApikIWlR,UNITN,95,UNITN,Female,Italian,Social Sciences,Social Sciences,21,BSc,...,83.333336,75.00,Italian,Psychology and Cognitive Science,Rovereto,NaN,3994.0,not very active,4.0,active
245,I8ggVucB5iLIW7C,UNITN,96,UNITN,Male,Italian,Social Sciences,Social Sciences,20,BSc,...,NaN,NaN,Italian,Sociology and Social Research,In another city away from where the university...,NaN,43811.0,very active,NaN,inactive
246,XpFgWbI8bvt4REU,UNITN,97,UNITN,Male,Italian,Engineering and Applied Sciences,Engineering and Applied Sciences,22,MSc,...,75.000000,56.25,Italian,Information Engineering and Computer Science,Trento,East hill (Mesiano/Povo/Oltrecastello/San Dona...,23515.0,active,5.0,active
247,nq8et8Sx7XXJ3P7,UNITN,98,UNITN,Male,Italian,Natural Sciences,Natural Sciences,22,BSc,...,75.000000,37.50,Italian,Physics,Trento,Trento South (Bolghera/Clarina/San Bartolomeo/...,NaN,NaN,12.0,very active


In [69]:
# step activity reclassification: we already know active and inactive users, so we do not have information on those users who didn't have the sensor on
complete_dataset['step activity'] = pd.Categorical(complete_dataset['step activity'])
step_validusers = complete_dataset['step activity'].dropna()
print(f"we have {step_validusers.shape[0]} valid users from the step activity dataset")

we have 124 valid users from the step activity dataset


### Saving the data for later analysis

In [70]:
complete_dataset = complete_dataset[['userid', 'w1_A01', 'Dep_UNITN', 'degree', 'w1_A04UNITN', 'w1_A10UNITN', 'step activity', 'sport activity']]

In [71]:
%%timeit
complete_dataset.to_csv("data/step_sport_activity.csv")

3.65 ms ± 345 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
